In [218]:
import pandas as pd
import numpy as np
#from scipy.stats import beta # Beta probability function
from scipy.special import beta # Beta raw function
import os
import math

In [159]:
os.chdir('C:/Users/renan/Desktop/inequality-segregation-supplementary-files/')

In [160]:
census_2010 = pd.read_csv('data/std_2010_fullcount.csv', encoding = "ISO-8859-1", sep = ",")
df = census_2010.loc[census_2010.county == "Riverside County"][['trtid10','tract','pop10','nhblk10']]
df.sort_values('nhblk10').head()

,trtid10,tract,pop10,nhblk10
6073,6065980004,Census Tract 9800.04,9,0
12813,6065940100,Census Tract 9401,166,3
12741,6065045704,Census Tract 457.04,2804,4
12689,6065045124,Census Tract 451.24,1505,9
6061,6065041410,Census Tract 414.10,3288,9


In [161]:
# M_set = {m_1, ..., m_r} 
M_set = [1, 2, 3, 4, 5, 6, 6242]

# Assuming c Beta distributions
c = 2

In [177]:
n =  0 # 3 #677
m =  9 # 166 #M_set[6]

# Count the number of units of size m with n minority individuals
A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())

def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
    result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
    return result

lambdas = [0.1, 0.9]
alphas  = [0.5, 0.45]
betas   = [0.7, 0.1]

inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()

A_n_m * np.log(inner_sum)

-2.520981363054838

In [178]:
list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))

[0.023005810535624515, 0.05737487486002151]

In [164]:
#for m in M_set:

def evaluate_likelihood_for_n(n):
    A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())

    def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
        result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
        return result

    inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()

    return A_n_m * np.log(inner_sum)

In [165]:
m1 = 5

range(m1)

list(map(lambda x: evaluate_likelihood_for_n(x), range(m1)))

[-2.520981363054838, -0.0, -0.0, -0.0, -0.0]

In [246]:
lambdas = [0.1, 0.9]
alphas  = [0.5, 0.45]
betas   = [0.7, 0.1]

def evaluate_likelihood_for_m(m):
    
    def evaluate_likelihood_for_n(n):

        def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
            result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
            return result

        A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())
        #inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()
        inner_sum = np.nansum(np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))))
        
        #if np.isnan(np.log(inner_sum)):
        #    return 0
        #else:
        #    return A_n_m * np.log(inner_sum) # Add some noise + 0.001 to avoid 0 evaluation in the log
        try:
            return A_n_m * np.log(inner_sum)
        except:
            return 0
    
    aux = range(m)
    #outer_sum = np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))).sum()
    outer_sum = np.nansum(np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))))
    return outer_sum

In [222]:
evaluate_likelihood_for_m(9)

-2.520981363054838

In [223]:
evaluate_likelihood_for_m(166)

-18.663144501908384

In [236]:
np.isnan(na)

True

In [249]:
M_set = [9, 166, 1100, 2500, 10000]

np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum()

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


-21.18412586496322

In [252]:
def evaluate_likelihood_for_M_set(m_set):
    # m_set is a list of m's
    return np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum()

In [253]:
evaluate_likelihood_for_M_set(M_set)

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


-21.18412586496322

In [143]:
lambdas = [0.1, 0.9]
alphas  = [0.5, 0.45]
betas   = [0.7, 0.1]

def evaluate_likelihood_for_n_and_m(n, m):
    
    A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())

    def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
        result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
        return result

    inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()
    return A_n_m * np.log(inner_sum)
    
#    aux = range(m)
#    outer_sum = np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))).sum()
#    return outer_sum

In [216]:
na = evaluate_likelihood_for_n_and_m(9, 0)
na

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  


nan